# Computing Recovery Metrics with `spectral_recovery`

This notebook provides a quick, interactive, example for how to start using the `spectral_recovery` package for computing recovery metrics. Specifically, this notebook will work through the steps:

1. Reading in timeseries data
2. Computing indices
3. Defining a restoration site
4. Deriving a recovery target
5. Plotting a spectral trajectory
6. Computing recovery metrics

For a more detail explanation of each step and the functions/methods they use, see the documentation page.

In [ ]:
!pip install spectral_recovery

Once the tool is installed, you can import the package:

In [ ]:
import spectral_recovery as sr
import numpy as np

## Reading in Timeseries Data and Computing Indices

The set of annual composites that we'll be using in this notebook was created using the [GEE BAP tool](https://github.com/saveriofrancini/bap) and is available in the project repository's test_data directory.

In [ ]:
timeseries = sr.read_timeseries(
    "test_data/annual_composites/landsat",
    band_names={1: "blue", 2: "green", 3: "red", 4: "nir", 5: "swir16", 6: "swir22"},
    array_type="dask"
)
timeseries

If your rasters uses 0 to represent Null data please set these values to NaN before proceeding. The `spectral_recovery` tool expects Null data to be NaN not 0. Leaving 0 values will lead to unexpected behaviour.

In [ ]:
timeseries = timeseries.where(timeseries != 0.0, np.nan)

When selecting indices you should select the indices which best align with your recovery/restoration goals. For the sake of this example, we will simply compute NBR, NDVI, and SAVI. See the [description of the core indices] in the theoretical basis document for more information on selecting indices.

In [ ]:
indices = sr.compute_indices(timeseries, indices=["NBR", "NDVI", "SAVI"])
indices

## Defining Restoration Site

In this example we'll look at a site of a forest fire that occured in 2005 in Northern British Columbia within the unceeded territory of Saik'uz first nation.

In [ ]:
restoration_site = sr.read_restoration_polygons(
    "test_data/wildfire_516.gpkg",
    disturbance_start="2005",
    restoration_start="2006"
)
restoration_site

### Computing Recovery Metrics

Now that we've prepped our index timeseries data and restoration site, we can derive recovery targets:

In [ ]:
hist_polygon_median = sr.targets.median_target(
    polygon=restoration_site, 
    timeseries_data=indices, 
    reference_start="2003", 
    reference_end="2004",
    scale="polygon",
)
hist_polygon_median


### Plotting Spectral Trajectory of Restoration Site

In [ ]:
sr.plot_spectral_trajectory(
    timeseries_data=indices,
    restoration_polygons=restoration_site,
    recovery_target=hist_polygon_median,
    reference_start="2003", 
    reference_end="2004", 
)

## Computing Recovery Metrics

In [ ]:
metrics = sr.compute_metrics(
    metrics=["Y2R", "dNBR", "R80P", "YrYr"],
    restoration_polygons=restoration_site,
    timeseries_data=indices,
    recovery_target=hist_polygon_median,
    timestep=4
)
metrics

## Viewing Recovery Metrics

In [ ]:
import matplotlib.pyplot as plt

def plot_metrics(metric):
    """ Plot the three bands in a single figure. """
    fig, axes = plt.subplots(ncols=3, sharex=True, sharey=True, figsize=[15, 4])
    for i in range(3):
        metric[0,i,:,:].plot(ax=axes[i])
        axes[i].set_title(f"{str(metric.metric.values[0])}: {metric['band'].values[i]}")
        axes[i].set_xlabel("X coordinate")
        if i == 0:
            axes[i].set_ylabel("Y coordinate")
        else:
            axes[i].set_ylabel("")
        plt.tight_layout()


Once you've computed your desired metrics, you can start visualization and analysis.

### Y2R

In [ ]:
y2r = metrics.sel(metric=["Y2R"])
y2r_unrecovered_changed = y2r.where(y2r != -9999, -10)
plot_metrics(y2r_unrecovered_changed)

Notice that before plotting Y2R we changed -9999 values to -10. Any Y2R pixels that have -9999 values are pixels that have not yet recovered relative to the recovery target. We changed the value to make visualizations easier.

### R80P

In [ ]:
r80p = metrics.sel(metric=["R80P"])
plot_metrics(r80p)

### YrYr

In [ ]:
yryr = metrics.sel(metric=["YrYr"])
plot_metrics(yryr)

### dNBR

In [ ]:
dNBR = metrics.sel(metric=["dNBR"])
plot_metrics(dNBR)

###  Writing Results

To write your metric results to raster files, the simpliest way is to use `rioxarray`'s `to_raster` function:


In [ ]:
# write out y2r results to file:
y2r.sel(metric="Y2R").rio.to_raster("./y2r.tif")